<a href="https://colab.research.google.com/github/joris-vaneyghen/mss-jazz-playalong/blob/main/segmentation/cluster_segments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# download our audio example
!git clone https://github.com/joris-vaneyghen/mss-jazz-playalong.git

In [ ]:
import numpy as np
import json
import os

from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score

from sklearn.decomposition import PCA

import matplotlib.pyplot as plt



In [ ]:
with open('mss-jazz-playalong/sound_classes_audioset.json', 'r') as f:
  sound_classes_audioset = json.load(f)

In [ ]:
input_path = 'mss-jazz-playalong/examples'
output_path = 'mss-jazz-playalong/out/segment_and_tag'
resolution = 0.32 # resolution of EfficientAT model

In [ ]:
def load_json(dir, mp3_file):
    # Replace .mp3 extension with .json
    json_file_name = mp3_file.replace('.mp3', '.json')
    file_path = os.path.join(dir, json_file_name)

    # Check if the .json file exists
    if not os.path.exists(file_path):
        return {}  # Return an empty dictionary if the .json file doesn't exist

    # Load the JSON file if it exists
    with open(file_path, 'r') as file:
        return json.load(file)

# def save_json(dir, mp3_file, data):
#     # Replace .mp3 extension with .json
#     json_file_name = mp3_file.replace('.mp3', '.json')
#     file_path = os.path.join(dir, json_file_name)

#     # Check if directory exists, create it if not
#     if not os.path.exists(dir):
#         os.makedirs(dir)

#     # Save the data to the .json file
#     with open(file_path, 'w') as file:
#         json.dump(data, file, indent=4)

def iterate_files(dir):
    for file_name in os.listdir(dir):
        if file_name.endswith('.mp3'):
            yield file_name


def calc_features(segment_preds, drums, bass, vocals):
  features = []
  for audio_class in sound_classes_audioset:
    if audio_class == 'drums':
      features.append(drums)
    elif audio_class == 'bass':
      features.append(bass)
    elif audio_class == 'vocals':
      features.append(vocals)
    else:
      audioset_indices = [ audio_set_cls['index'] for audio_set_cls in sound_classes_audioset[audio_class]]
      features.append(segment_preds[:, audioset_indices].sum(axis=1))

  # return np.array(features)
  return np.stack(features, axis=1)


In [ ]:
segment_lengths =  []
segment_preds = []
drums = []
bass = []
other = []
vocals = []
for mp3_file in iterate_files(input_path):
  data = load_json(output_path, mp3_file)
  if ('demucs' in data.keys() and 'segments' in data.keys()):
    segments = data['segments']
    for segment in segments:
      length = segment['end_idx'] - segment['start_idx']
      segment_lengths.append(length)
      segment_preds.append(segment['preds'])
      drums.append(segment['drums'])
      bass.append(segment['bass'])
      other.append(segment['other'])
      vocals.append(segment['vocals'])



In [ ]:
# plot frequency chart of segment_lengths
plt.hist(segment_lengths, bins=20)
plt.xlabel("Segment Length")
plt.ylabel("Frequency")
plt.title("Frequency Chart of Segment Lengths")
plt.show()


In [ ]:
segment_preds = np.array(segment_preds)
drums = np.array(drums)
bass = np.array(bass)
other = np.array(other)
vocals = np.array(vocals)
demucs_features = np.stack((drums, bass, vocals, other), axis=1)

In [ ]:
features = calc_features(segment_preds, drums, bass, vocals);
features.shape

In [ ]:
# normalize features
normalized_features = (features - np.mean(features, axis=0, keepdims=True)) / np.std(features, axis=0, keepdims=True)

In [ ]:
def find_best_cluster(data):
  eps_range = np.arange(0.1, 10, 0.1)  # Example range of epsilon values
  min_samples = data.shape[0] // 100

  best_silhouette_score = -1
  silhouette_scores = []
  nb_labels = []
  nb_noise = []
  best_eps = None
  best_labels = None

  for eps in eps_range:
    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    labels = dbscan.fit_predict(data)



    unique_labels, label_counts = np.unique(labels, return_counts=True)
    best_label = unique_labels[np.argmax(label_counts)]

    # Ignore noisy samples (-1) for silhouette calculation
    if len(np.unique(labels)) > 2 and best_label != -1:
      data_without_noise = data[labels != -1]
      labels_without_noise = labels[labels != -1]

      silhouette_avg = silhouette_score(data_without_noise, labels_without_noise, metric='euclidean')
      silhouette_scores.append(silhouette_avg)
      nb_labels.append(len(np.unique(labels)))
      nb_noise.append(np.count_nonzero(labels == -1))

      if silhouette_avg > best_silhouette_score:
        best_silhouette_score = silhouette_avg
        best_eps = eps
        best_labels = labels
    else:
      silhouette_scores.append(-1)
      nb_labels.append(-1)
      nb_noise.append(-1)

  print(f"Best epsilon: {best_eps}")
  print(f"Best Silhouette Score: {best_silhouette_score}")

  # Perform DBSCAN clustering with the best parameters
  dbscan = DBSCAN(eps=best_eps, min_samples=min_samples)
  labels = dbscan.fit_predict(data)

  # Print cluster labels for each sample
  print("Cluster labels for the data points:", labels)

  unique_labels, label_counts = np.unique(labels, return_counts=True)
  best_label = unique_labels[np.argmax(label_counts)]
  print(f"Cluster label with max count: {best_label}")

  # Plot the frequencies of labels
  plt.bar(unique_labels, label_counts)
  plt.xlabel('Cluster Labels')
  plt.ylabel('Frequency')
  plt.title('Frequency of Cluster Labels')
  plt.show()

  # Plot Silhouette Score for each k
  plt.figure(figsize=(10, 5))
  plt.plot(eps_range, silhouette_scores, 'bx-')
  plt.xlabel('eps')
  plt.ylabel('Silhouette Score')
  plt.title('Silhouette Score for Different eps')
  plt.show()

  plt.figure(figsize=(10, 5))
  plt.plot(eps_range, nb_labels, 'bx-')
  plt.xlabel('eps')
  plt.ylabel('Nb labels')
  plt.title('Nb labels for Different eps')
  plt.show()

  plt.figure(figsize=(10, 5))
  plt.plot(eps_range, nb_noise, 'bx-')
  plt.xlabel('eps')
  plt.ylabel('Nb noise')
  plt.title('Nb noise for Different eps')
  plt.show()

  indexes_best_cluster = np.where(labels == best_label)[0]


  return indexes_best_cluster



In [ ]:
indexes_best_cluster = find_best_cluster(normalized_features)
features_best_cluster = features[indexes_best_cluster]
features_non_best_cluster = features[~np.isin(np.arange(features.shape[0]), indexes_best_cluster)]

In [ ]:
from sklearn.mixture import GaussianMixture
from scipy.stats import chi2


In [ ]:
n_components = 8
gmm = GaussianMixture(n_components=n_components, covariance_type='full')
gmm.fit(features_best_cluster)


In [ ]:
X = features_best_cluster

# Test verschillende aantallen componenten
n_components_range = range(1, 11)  # Test van 1 tot 10 componenten
aic_scores = []
bic_scores = []

for n_components in n_components_range:
    gmm = GaussianMixture(n_components=n_components, covariance_type='full', random_state=42)
    gmm.fit(X)

    # Voeg AIC en BIC toe aan de lijsten
    aic_scores.append(gmm.aic(X))
    bic_scores.append(gmm.bic(X))

# Plot de AIC en BIC scores
plt.figure(figsize=(8, 6))
plt.plot(n_components_range, aic_scores, label='AIC', marker='o')
plt.plot(n_components_range, bic_scores, label='BIC', marker='o')
plt.xlabel('Aantal componenten')
plt.ylabel('Score')
plt.title('AIC en BIC om het beste aantal componenten te bepalen')
plt.legend()
plt.show()


In [145]:
X = features_best_cluster

# log_likelihood = gmm.score_samples(features_best_cluster)


# Bereken de Mahalanobis distance per component
mahalanobis_distances = []
for i in range(gmm.n_components):
    diff = X - gmm.means_[i]
    inv_cov = np.linalg.inv(gmm.covariances_[i])
    mahalanobis_distances.append(np.sum(np.dot(diff, inv_cov) * diff, axis=1))

# Neem de minimale Mahalanobis afstand voor elk punt (dichtstbijzijnde component)
mahalanobis_distance = np.min(mahalanobis_distances, axis=0)

# Chi2 threshold bij 95% confidence voor het aantal dimensies
p_value_threshold = 0.90
chi2_threshold = chi2.ppf(p_value_threshold, df=X.shape[1])

# Tel de punten binnen en buiten de chi2 threshold
nb_in  = np.count_nonzero(mahalanobis_distance <= chi2_threshold)
nb_out = np.count_nonzero(mahalanobis_distance > chi2_threshold)

print(f"Aantal punten binnen de 95% drempel: {nb_in}")
print(f"Aantal punten buiten de 95% drempel: {nb_out}")


Aantal punten binnen de 95% drempel: 195
Aantal punten buiten de 95% drempel: 14


In [ ]:
# Range of clusters to try (from 2 to 5)
cluster_range = range(2, 40)

# List to store SSE (sum of squared distances) for the elbow method
sse = []
silhouette_scores = []


# pca = PCA(n_components=20)  # Reduce to 50 dimensions or fewer


# combined = np.concatenate((segment_preds, demucs_features * 10 ), axis=1)
# data = pca.fit_transform(combined)
data = normalized_features

# Perform KMeans clustering for different values of k
for k in cluster_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(data)
    sse.append(kmeans.inertia_)  # SSE for elbow method
    silhouette_avg = silhouette_score(data, kmeans.labels_)
    silhouette_scores.append(silhouette_avg)

# Plot SSE for elbow method
plt.figure(figsize=(10, 5))
plt.plot(cluster_range, sse, 'bx-')
plt.xlabel('Number of clusters (k)')
plt.ylabel('SSE (Sum of Squared Distances)')
plt.title('Elbow Method for Optimal k')
plt.show()

# Plot Silhouette Score for each k
plt.figure(figsize=(10, 5))
plt.plot(cluster_range, silhouette_scores, 'bx-')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Score for Different k')
plt.show()

# Choose the best k based on visual inspection of the elbow and silhouette score
best_k = cluster_range[np.argmax(silhouette_scores)]
print(f"Best number of clusters: {best_k}")

# Perform KMeans clustering with the best k
kmeans = KMeans(n_clusters=best_k, random_state=42)
labels = kmeans.fit_predict(data)

# Print cluster labels for each sample
print("Cluster labels for the data points:", labels)

unique_labels, label_counts = np.unique(labels, return_counts=True)

# Plot the frequencies of labels
plt.bar(unique_labels, label_counts)
plt.xlabel('Cluster Labels')
plt.ylabel('Frequency')
plt.title('Frequency of Cluster Labels')
plt.show()


In [ ]:
# cluster data=normalized_features now with DBSCAN instead of kmeans



# Data is already normalized_features

# Perform DBSCAN clustering with different values of eps and min_samples
eps_range = np.arange(0.1, 10, 0.1)  # Example range of epsilon values
min_samples_range = range(3, 5)  # Example range of min_samples values

best_silhouette_score = -1
best_eps = None
best_min_samples = None
best_labels = None

for eps in eps_range:
    for min_samples in min_samples_range:
        dbscan = DBSCAN(eps=eps, min_samples=min_samples)
        labels = dbscan.fit_predict(data)

        # Ignore noisy samples (-1) for silhouette calculation
        if len(np.unique(labels)) > 1:
            silhouette_avg = silhouette_score(data, labels, metric='euclidean')

            if silhouette_avg > best_silhouette_score:
                best_silhouette_score = silhouette_avg
                best_eps = eps
                best_min_samples = min_samples
                best_labels = labels

print(f"Best epsilon: {best_eps}")
print(f"Best min_samples: {best_min_samples}")
print(f"Best Silhouette Score: {best_silhouette_score}")

# Perform DBSCAN clustering with the best parameters
dbscan = DBSCAN(eps=best_eps, min_samples=best_min_samples)
labels = dbscan.fit_predict(data)

# Print cluster labels for each sample
print("Cluster labels for the data points:", labels)

unique_labels, label_counts = np.unique(labels, return_counts=True)

# Plot the frequencies of labels
plt.bar(unique_labels, label_counts)
plt.xlabel('Cluster Labels')
plt.ylabel('Frequency')
plt.title('Frequency of Cluster Labels')
plt.show()


In [ ]:
# for each clustercenter print largest dimensions. Use the dimension_names

dimension_names = [audio_class for  audio_class in sound_classes_audioset]

for i, cluster_center in enumerate(kmeans.cluster_centers_):
  top_indices = np.argsort(cluster_center)[::-1]  # Sort in descending order
  print(f"\nCluster {i + 1}: Top Dimensions")
  for j in range(5):  # Print top 5 dimensions
    dimension_index = top_indices[j]
    print(f"  {dimension_names[dimension_index]}: {cluster_center[dimension_index]:.4f}")


In [ ]:
import pickle

# Save the PCA model to a file
def save_pca_model(pca, filename="pca_model.pkl"):
  with open(filename, "wb") as f:
    pickle.dump(pca, f)

# Load the PCA model from a file
def load_pca_model(filename="pca_model.pkl"):
  with open(filename, "rb") as f:
    return pickle.load(f)

# Project new data using the loaded PCA model
def transform_data(pca_model, new_data):
  return pca_model.transform(new_data)

def save_kmeans_model(kmeans, filename="kmeans_model.pkl"):
  with open(filename, "wb") as f:
    pickle.dump(kmeans, f)

# Load the KMeans model from a file
def load_kmeans_model(filename="kmeans_model.pkl"):
  with open(filename, "rb") as f:
    return pickle.load(f)

# Predict cluster labels for new data using the loaded KMeans model
def predict_cluster_labels(kmeans_model, new_data):
  return kmeans_model.predict(new_data)

# Example usage:

# Assuming 'pca' is your trained PCA model
save_pca_model(pca)

# Assuming 'kmeans' is your trained KMeans model
save_kmeans_model(kmeans)

# Later, to load and use the model:
loaded_pca = load_pca_model()

# Later, to load and use the model:
loaded_kmeans = load_kmeans_model()

# Example new data
new_data = np.random.rand(10, 531) # Replace with your actual new data

# Project the new data using the loaded PCA model
transformed_data = transform_data(loaded_pca, new_data)
print(transformed_data.shape)

# Predict the cluster labels for the new data
new_labels = predict_cluster_labels(loaded_kmeans, transformed_data)
print(new_labels)